# Analysis: Binding Free energies

## Functions

In [1]:
# Loads stuff
import sys
sys.path.insert(0,"/home/lg3u19/OnePy")
import onetep_v0_1 as op
import pathlib
import pandas as pd
pd.set_option('display.precision',3) 
import copy

In [2]:
def cavity_correction(E_host_non_pol,E_complex_non_polar):
    return 7.116*(E_host_non_pol-E_complex_non_polar)


In [3]:
def binding_free_energy(complex_obj,host_obj,ligand_obj,units='Ha',host_cavity_correction=True,entropy=False):
    delta_E = complex_obj.total_energy_vac - host_obj.total_energy_vac - ligand_obj.total_energy_vac
    delta_solvation = complex_obj.total_solvation_energy - host_obj.total_solvation_energy - ligand_obj.total_solvation_energy
    
    if host_cavity_correction==True:
        cavity_corr = cavity_correction(host_obj.total_apolar_energy,complex_obj.total_apolar_energy)
        delta_solvation_corrected = delta_solvation + cavity_corr
    else:
        delta_solvation_corrected=delta_solvation
    if entropy==False:   
        binding_free_energy = delta_E + delta_solvation_corrected
    else:
        binding_free_energy = delta_E + delta_solvation_corrected - entropy
     
    if units=='Ha':
        return delta_E,delta_solvation,delta_solvation_corrected,entropy,binding_free_energy
    elif units=='kcal/mol':
        return op.hartree_to_kcal_mol(delta_E),op.hartree_to_kcal_mol(delta_solvation),\
               op.hartree_to_kcal_mol(delta_solvation_corrected),\
               op.hartree_to_kcal_mol(entropy),op.hartree_to_kcal_mol(binding_free_energy)

In [4]:
def exclude_function(snapshot,functional):
    """ Functions that returns total energy in vac, solvation enerygm, and apolar energy
    for given snapshots and functionals from data dictionary with pre-filled values"""
    if functional == False:
        print ('Needs to know functional')
        return False
    data = {'BLYP':{'complex_17201':{'E_vac':-11703.28810914757196,
                             'E_solv':-11707.17781111132354,
                             'apolar':2.33371847090407+-1.67776855169471},
                    'host_17201':{'E_vac':-11642.52098752837628,
                             'E_solv':-11646.40564469317906,
                             'apolar':2.35343919310646+-1.69194627190406},
                    'complex_24801':{'E_vac':-11703.26093899935950,
                             'E_solv':-11707.23062178841792,
                             'apolar':2.34244656224695+-1.68404339476339},
                    'host_24801':{'E_vac':-11642.50372680707369,
                             'E_solv':-11646.47373658255674,
                             'apolar':2.35905073333339+-1.69598054846171},
                    'complex_32401':{'E_vac':-11703.48681972792838,
                             'E_solv':-11707.25277486920822,
                             'apolar':2.36436511707726+-1.69980119179477}
                   },
            'VV10':{'complex_24801':{'E_vac':-11755.75196689821314,
                             'E_solv':-11759.71220481162891,
                             'apolar':2.27888540828097+-1.63834769214840},
                    'host_24801':{'E_vac':-11694.67952414450701,
                             'E_solv':-11698.63966000627261,
                             'apolar':2.29724655713729+-1.65154798108993},
                    'complex_32401':{'E_vac':-11755.99253040028270,
                             'E_solv':-11759.76183088717517,
                             'apolar':2.29984764129358+-1.65341796551699},
                    'host_32401':{'E_vac':-11694.93358778918628,
                             'E_solv':-11698.69660710869357,
                             'apolar':2.32170136252029+-1.66912915204990}
                    }}
  
    return data[functional][snapshot]['E_vac'],data[functional][snapshot]['E_solv']-data[functional][snapshot]['E_vac'],data[functional][snapshot]['apolar']
    

In [5]:
def get_average_binding_manual(outfile_dir,correction=True,exclude_list=[],functional=False,
                        exclude_function=False,snapshots=False): 
    """ Given a directory of outfiles, returns df with relevant values for QMPBSA as 
    well as series object with averages over all snaphsots of these values"""
    # create dictionary of onetep objects for each outfile
    object_dict = op.load_out_files(outfile_dir,format_flag=True,
                                    delim='_',split_num=0)
    
    
    # determine which snapshots to process
    if snapshots==False: # if false, use all snapshots present
        # find out which snapshots we have
        snapshots = []
        for key in object_dict.keys():
            snapshot = key.split(sep='_')[-1]
            if snapshot not in snapshots:
                snapshots.append(snapshot)
    else: # if not false, a list of snapshots was passed
        pass
        
    # laod data from onetep objects 
    # allows for excluded snapshots for some functionals for which a separate
    # function is called to assign key values determine in advance from 
    # output files or restarted calculations
    # exclude list must have accurate file names
    for key in object_dict.keys():
        object_dict[key].get_atom_counts()
        object_dict[key].get_input_flags()
        object_dict[key].get_dispersion()
        if key in exclude_list:
            vac,solvation,apolar = exclude_function(key,functional)
            object_dict[key].total_energy_vac = vac
            object_dict[key].total_solvation_energy = solvation
            object_dict[key].total_apolar_energy = apolar
            
        else:
            object_dict[key].check_is_auto_solvation()
            object_dict[key].get_total_time()
            object_dict[key].get_energy_conv()
            object_dict[key].get_solvation_summary_new()
        
    # generate a df with values of interest
    temp_dir = {}
    for snapshot in snapshots:
        temp_dir[snapshot]=binding_free_energy(object_dict['complex_'+snapshot],
                                                          object_dict['host_'+snapshot],
                                                          object_dict['ligand_'+snapshot],
                                                          units='kcal/mol',host_cavity_correction=correction)
    object_dict_bind = pd.DataFrame.from_dict(temp_dir,orient='index',
                                              columns=['E','G_solv','G_solv_corrected','S','G_bind'])
    object_dict_bind.drop(labels=['S'],axis='columns',inplace=True)
    
    return object_dict_bind, object_dict_bind.mean()

In [6]:
def get_average_binding(outfile_dir,correction=True,snapshots=False): 
    """ Given a directory of outfiles, returns df with relevant values for QMPBSA as 
    well as series object with averages over all snaphsots of these values"""
    # create dictionary of onetep objects for each outfile
    object_dict = op.load_out_files(outfile_dir,format_flag=True,
                                    delim='_',split_num=0)
    # laod data from onetep objects 
    for key in object_dict.keys():
        object_dict[key].check_is_auto_solvation()
        object_dict[key].get_total_time()
        object_dict[key].get_atom_counts()
        object_dict[key].get_energy_conv()
        object_dict[key].get_input_flags()
        object_dict[key].get_solvation_summary_new()
        object_dict[key].get_dispersion()
        
    # determine which snapshots to process
    if snapshots==False: # if false, use all snapshots present
        # find out which snapshots we have
        snapshots = []
        for key in object_dict.keys():
            snapshot = key.split(sep='_')[-1]
            if snapshot not in snapshots:
                snapshots.append(snapshot)
    else: # if not false, a list of snapshots was passed
        pass
        

    # generate a df with values of interest
    temp_dir = {}
    for snapshot in snapshots:
        temp_dir[snapshot]=binding_free_energy(object_dict['complex_'+snapshot],
                                                          object_dict['host_'+snapshot],
                                                          object_dict['ligand_'+snapshot],
                                                          units='kcal/mol',host_cavity_correction=correction)
    object_dict_bind = pd.DataFrame.from_dict(temp_dir,orient='index',
                                              columns=['E','G_solv','G_solv_corrected','S','G_bind'])
    object_dict_bind.drop(labels=['S'],axis='columns',inplace=True)
    
    return object_dict_bind, object_dict_bind.mean()

In [7]:
def load_data(outfile_dir_root,functionals,correction=False,snapshots=False):
    """ given directory of dat files, gets avera binding energies with or without cavity correction """
    list_of_mean_series = []
    for functional in functionals:
        outfile_dir = outfile_dir_root / functional
        summary_df, mean_series = get_average_binding(outfile_dir,correction=correction,snapshots=snapshots)
        mean_series.name = functional
        list_of_mean_series.append(mean_series)
    return pd.concat(list_of_mean_series,axis=1) 
    

In [50]:
def load_data_all(outfile_dir_root,functionals,correction=False,snapshots=False):
    """ given directory of dat files, gets avera binding energies with or without 
        cavity correction and also returns raw data """
    list_of_mean_series = []
    dict_of_summary_df = {}
    for functional in functionals:
        outfile_dir = outfile_dir_root / functional
        summary_df, mean_series = get_average_binding(outfile_dir,correction=correction,snapshots=snapshots)
        mean_series.name = functional
        list_of_mean_series.append(mean_series)
        dict_of_summary_df[functional] = summary_df
    return pd.concat(list_of_mean_series,axis=1) , dict_of_summary_df
    

# Functions to deal with dispersion

In [15]:
def load_disp_from_folder(root,exclude_list=[".","_",","],dat_file_name='disp.data'):
    """ goes through folders in root dir and loads csv files into dict by folder name
    folder correspond to ligands """
    root = pathlib.Path(root) 
    disp_dict = {}
    for folder in root.iterdir():
        if folder.is_dir():
            if folder.name[0] not in exclude_list:
                disp_dict[folder.name]=pd.read_csv(folder/dat_file_name,sep=' ')
    return disp_dict 

In [16]:
def process_disp(disp_dict,snapshots):
    """ return dictionary of mean net dispersion over a subset of snapshots"""
    mean_dict = {}
    for key in disp_dict.keys():
        # select subset of snaps
        subset = disp_dict[key].loc[[int(x) for x in snapshots],:]
        # calc net dispersion term
        subset['diff']=subset['complex']-subset['host']-subset['ligand']
        mean_dict[key]=subset.mean()['diff']
    return mean_dict

In [17]:
def elstner_dispersion_collection(directories, functional, snapshots=False):
    """ complicated way of getting the elstner dispersion terms for all onetep calcs, 
    these are processed to get the mean mean net dispersion over a set of snapshots_present
    directories: is a list of directories in which to find onetep outfiles
    Changed to take only one functional at a time as argument""" 
    data_temp = {}
    for outfile_dir_root in directories:
        inner_dict = {}
        outfile_dir = outfile_dir_root / functional
        # load a dictionary of onetep objects using OnePy 
        object_dict = op.load_out_files(outfile_dir,format_flag=True,
                                    delim='_',split_num=0) 
        # this appears to override the snapshots_present parameter, INVESTIGATE
        # determine which snapshots have been loaded for a given functional and dir
        snapshots_present = []
        for key in object_dict.keys():
            snapshot = key.split(sep='_')[-1]
            if snapshot not in snapshots_present:
                snapshots_present.append(snapshot)
        # call dispersion routine for each object
        for key in object_dict.keys():
            object_dict[key].get_dispersion()
        # initialize lists
        complex_disp=[]
        host_disp=[]
        ligand_disp=[]
        diff=[]
        # loop over all snapshots loaded from dir for this functional
        for snapshot in snapshots_present:
            # populate lists with comp, host, lig disp and calc net disp (diff)
            complex_disp.append(op.hartree_to_kcal_mol(object_dict['complex_'+snapshot].empirical_dispersion))
            host_disp.append(op.hartree_to_kcal_mol(object_dict['host_'+snapshot].empirical_dispersion))
            ligand_disp.append(op.hartree_to_kcal_mol(object_dict['ligand_'+snapshot].empirical_dispersion))
            diff.append(op.hartree_to_kcal_mol(object_dict['complex_'+snapshot].empirical_dispersion-
                                              object_dict['host_'+snapshot].empirical_dispersion-
                                              object_dict['ligand_'+snapshot].empirical_dispersion))
        # populates an inner loop temp dict with the data from above
        inner_dict['complex']=complex_disp
        inner_dict['host']=host_disp
        inner_dict['ligand']=ligand_disp
        inner_dict['diff']=diff
        # adds snapshots present and processed
        inner_dict['snapshot']=[int(x) for x in snapshots_present]
        # convert to df and set snapshots as indexz
        df_inner_dict=pd.DataFrame.from_dict(inner_dict)
        df_inner_dict=df_inner_dict.set_index('snapshot')
        # add the inner loop df to the outer loop dictionary with key as name of ligand
        data_temp[str(outfile_dir_root.name).split('_')[0]]=df_inner_dict
    # use OnePy utility to generate df from 2 times nested dict. Although is this even nested....
    #dispersion_df = op.df_from_dict_2nested(data_temp)    
    # call process_disp to get mean value over subset of snapshots. Here need to select subsets...
    if snapshots == False:
        mean_disp_elstner = process_disp(data_temp,snapshots_present)
    else:
        mean_disp_elstner = process_disp(data_temp,snapshots)
    return mean_disp_elstner

In [18]:
def d2_d3_dispersion_correction(ligands,snapshots,mean_disp_elstner,root_dir):
    """ streamlines process of getting rel mean dispersion corrections to augment final binding energies"""
    mean_disp_correction = {}
    for ligand in ligands:
        temp1= load_disp_from_folder(root_dir+ligand)
        temp2= process_disp(temp1,snapshots)
        temp2['elstner']=mean_disp_elstner[ligand]
        mean_disp_correction[ligand]=temp2
    df_mean_disp = pd.DataFrame.from_dict(mean_disp_correction)
    df_rel_mean_dips = df_mean_disp.copy()
    df_rel_mean_dips = df_rel_mean_dips.drop('phenol',axis=1)
    for ligand in ligands:
        if ligand != 'phenol':
            df_rel_mean_dips[ligand]=df_mean_disp[ligand]-df_mean_disp['phenol']
    return df_rel_mean_dips


In [19]:
def edit_pbe_df(df_binding_energies,df_rel_mean_disp,disp_list=['bj','bjm','old','zero','zero']):
    """ given the output df of norm rel binding energies, adds columns for the different empirical
    disp corrections to pbe by suptraction elstener and adding XXX """ 
    for disp in disp_list:
        df_binding_energies['PBE_'+disp]=df_binding_energies['PBE']-df_rel_mean_disp.transpose()['elstner']+df_rel_mean_disp.transpose()[disp]
    return df_binding_energies
                                                           

## Determine list of Snapshots present for phenol PBE, and define sets of snaps

In [20]:
snapshots_5 = ['24801', '32401', '17201', '9601','2001'] 
snapshots_10 = ['24801', '32401', '17201', '13401', '21001', '28601', '9601', '5801','36201','2001'] 
snapshots_50 = [2001, 2761, 3521, 4281, 5041, 5801, 6561, 7321, 8081, 8841, 9601, 10361, 11121, 11881, 12641, 13401, 14161, 14921, 15681, 16441, 17201, 17961, 18721, 19481, 20241, 21001, 21761, 22521, 23281, 24041, 24801, 25561, 26321, 27081, 27841, 28601, 29361, 30121, 30881, 31641, 32401, 33161, 33921, 34681, 35441, 36201, 36961, 37721, 38481, 39241]

In [21]:
# Load phernol PBE via OnePy and determine snapshots present
phenol_PBE = op.load_out_files("phenol_outfiles/PBE",
                               format_flag=True,
                               delim='_', split_num=0)
snapshots = []
for key in phenol_PBE.keys():
    snapshot = key.split(sep='_')[-1]
    if snapshot not in snapshots:
        snapshots.append(snapshot)
print (snapshots)

['24801', '32401', '17201', '13401', '21001', '28601', '9601', '5801', '36201', '2001']


# Collect entropy data

In [22]:
entropy_dict ={}
for ligand in ['phenol','methylphenol','catechol','fluoroaniline','hydroxyaniline']:
    entropy_data = pd.read_csv('entropy/'+ligand+'_entropy.txt',delimiter=' ',names=['snapshot','S'])
    entropy_data['snapshot'] = entropy_data['snapshot'].str.split('.').str[1]
    entropy_data = entropy_data.set_index('snapshot') 
    entropy_data['S']=entropy_data['S'].str.strip('a')
    entropy_data = entropy_data.apply(pd.to_numeric)
    entropy_dict[ligand]=entropy_data
entropy_data=False

In [23]:
def entropy_mean(entropy_data,snapshots):
    """ retruns mean over subset of snapshots for entropy, in future, all snaps must be 
    present , else key error raised..."""
    subset_df = entropy_data.loc[snapshots,:]
    return subset_df['S'].mean()

In [24]:
def entropy_relative(entropy_dict,snapshots,ligands,reference='phenol'):
    """ retruns series of mean entropies relative to reference """
    # get mean entropies over subset
    temp_dict_mean = {}
    for ligand1 in ligands:
        temp_dict_mean[ligand1] = entropy_mean(entropy_dict[ligand1],snapshots)
    # calc relative to reference entropy
    temp_dict_rel = {}
    for ligand2 in ligands:
        if ligand2!=reference:
            temp_dict_rel[ligand2] = temp_dict_mean[ligand2] - temp_dict_mean[reference]
    return pd.Series(temp_dict_rel)

In [25]:
ligands = ['phenol','methylphenol','catechol','fluoroaniline','hydroxyaniline']
entropy_50 = entropy_relative(entropy_dict,[str(x) for x in snapshots_50],ligands)
entropy_10 = entropy_relative(entropy_dict,[str(x) for x in snapshots_10],ligands)
entropy_5 = entropy_relative(entropy_dict,[str(x) for x in snapshots_5],ligands)


/Users/lennartgundelach/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1494: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


In [26]:
entropy_50

methylphenol     -1.553
catechol         -0.396
fluoroaniline    -1.155
hydroxyaniline   -1.013
dtype: float64

In [27]:
entropy_10

methylphenol      0.472
catechol          1.169
fluoroaniline     1.773
hydroxyaniline    2.168
dtype: float64

In [28]:
entropy_5

methylphenol      1.082
catechol          1.030
fluoroaniline     2.853
hydroxyaniline    2.282
dtype: float64

# Collect MM-PBSA Data

In [29]:
# Creat a dictionary in which each ligand has a DF of all data from .csv files for every snapshot
mm_dict = {}
for ligand in ['phenol','methylphenol','catechol','fluoroaniline','hydroxyaniline']:
    # load data from file
    mm_data = pd.read_csv('MM-PBSA/'+ligand+'/MM-energies-solvation.csv',delimiter=';')
    # set first column as index and relable to snaphot
    mm_data = mm_data.set_index(mm_data.columns[0])
    mm_data.index.name = 'snapshot'
    # select only snaphsots and get ridd of rest of file
    mm_data = mm_data.loc['2001':'39963',:]
    # drop empyt columns 
    mm_data = mm_data.dropna(axis=1)
    # deal with odd formatting of methylphenol file
    if 'Unnamed: 13' in mm_data.columns:
        mm_data = mm_data.drop('Unnamed: 13',axis=1)
    # rename columns
    mm_data.columns=['comp-gas','comp-polar',
                        'comp-non-polar','comp-total','host-gas',
                        'host-polar','host-non-polar','host-total',
                        'lig-gas','lig-polar','lig-non-polar',
                        'lig-total','net-gas','net-polar',
                        'net-non-polar','net-total']
    # change data type to numeric
    mm_data = mm_data.apply(pd.to_numeric)
    # add df to dictionary
    mm_dict[ligand]=mm_data
# make sure i dont accidenatlly use temp variable mm_data
mm_data = False
    

In [30]:
mm_dict['phenol']['net-tota-calc'] = mm_dict['phenol']['net-gas']+mm_dict['phenol']['net-polar']+mm_dict['phenol']['net-non-polar']

In [31]:
mm_dict['phenol']['net-calc-2'] = mm_dict['phenol']['comp-total'] - mm_dict['phenol']['host-total']-mm_dict['phenol']['lig-total']
#mm_dict['phenol']

In [32]:
subset_test=mm_dict['phenol'].loc[[str(x) for x in snapshots_50],:]
subset_test['correction']=7.116*(subset_test['host-non-polar']-subset_test['comp-non-polar'])
#subset_test

In [33]:
def mm_binding_energy(mm_data,snapshots,correction=True,entropy=False):
    """ given df with mm data on all snapshots, calcs mean binding eneryg 
    with our without cavity correction for a specified set of snapshots, 
    option to extend with entropy """
    # select subset 
    if snapshots=='All':
        subset_df = mm_data.loc[:,:]
    else:
        subset_df = mm_data.loc[snapshots,:]
    # calc mean binding energy depending on if cav correction enabled
    if correction == False:
        binding_energy = subset_df['net-total'].mean()
    elif correction == True:
        # determine mm cav correction term
        subset_df['correction']=2*(subset_df['host-non-polar']-subset_df['comp-non-polar'])
        subset_df['binding']=subset_df['net-total']+subset_df['correction']
        binding_energy = subset_df['binding'].mean()
    return binding_energy

In [34]:
def mm_binding_energy_gas(mm_data,snapshots,entropy=False):
    """ given df with mm data on all snapshots, calcs mean binding eneryg 
    with our without cavity correction for a specified set of snapshots, 
    option to extend with entropy """
    # select subset 
    if snapshots=='All':
        subset_df = mm_data.loc[:,:]
    else:
        subset_df = mm_data.loc[snapshots,:]
    # calc mean binding energy depending on if cav correction enabled
    binding_energy = subset_df['net-gas'].mean()
    return binding_energy

In [35]:
def mm_binding_energy_series(mm_dict,snapshots,ligands,correction=True,entropy=False,reference='phenol',ref_exp=-5.6):
    """ Given dict of mm data for each ligand, returns series with relative normalized binding energies relatibve to
    some reference, default phenol, with exp value of -5.6 kcal/mol. """
    # first get 'abs' binding energy for each ligand
    temp_dict_mean = {}
    for ligand1 in ligands: 
        temp_dict_mean[ligand1]=mm_binding_energy(mm_dict[ligand1],snapshots,correction,entropy)
        #temp_dict_mean[ligand1]=mm_binding_energy_gas(mm_dict[ligand1],snapshots,entropy)
    # det relative binding energy for non-ref ligands
    temp_dict_mean_rel = {}
    # calc normalization factor
    norm = ref_exp - temp_dict_mean[reference]
    for ligand2 in ligands:
        if ligand2 != reference:
            # apply norm to all non-ref ligands
            temp_dict_mean_rel[ligand2]=norm + temp_dict_mean[ligand2]
    # convert to series and return
    return pd.Series(temp_dict_mean_rel)

In [36]:
ligands = ['phenol','methylphenol','catechol','fluoroaniline','hydroxyaniline']
mm_5_uncorrected = mm_binding_energy_series(mm_dict,[str(x) for x in snapshots_5],ligands,correction=False)
mm_5_corrected = mm_binding_energy_series(mm_dict,[str(x) for x in snapshots_5],ligands,correction=True)
mm_10_uncorrected = mm_binding_energy_series(mm_dict,[str(x) for x in snapshots_10],ligands,correction=False)
mm_10_corrected = mm_binding_energy_series(mm_dict,[str(x) for x in snapshots_10],ligands,correction=True)
mm_50_uncorrected = mm_binding_energy_series(mm_dict,[str(x) for x in snapshots_50],ligands,correction=False)
mm_50_corrected = mm_binding_energy_series(mm_dict,[str(x) for x in snapshots_50],ligands,correction=True)

In [37]:
mm_5_corrected

methylphenol      -6.994
catechol          -5.086
fluoroaniline    -10.128
hydroxyaniline    -8.166
dtype: float64

In [38]:
mm_50_corrected

methylphenol     -7.803
catechol         -4.744
fluoroaniline    -9.164
hydroxyaniline   -7.737
dtype: float64

In [39]:
mm_50_uncorrected

methylphenol     -7.989
catechol         -4.760
fluoroaniline    -9.287
hydroxyaniline   -7.786
dtype: float64

In [40]:

mm_all_corrected = mm_binding_energy_series(mm_dict,'All',ligands,correction=True)
mm_all_corrected

methylphenol     -7.433
catechol         -4.147
fluoroaniline    -8.923
hydroxyaniline   -7.230
dtype: float64

In [41]:
mm_all_uncorrected = mm_binding_energy_series(mm_dict,'All',ligands,correction=False)
mm_all_uncorrected

methylphenol     -7.620
catechol         -4.150
fluoroaniline    -9.026
hydroxyaniline   -7.262
dtype: float64

# 10 Snaps

In [42]:
snapshots_10 = ['24801', '32401', '17201', '13401', '21001', '28601', '9601', '5801','36201','2001'] 

## Dispersion, 10 snaps

In [43]:
# loading elstner data
functionals = 'PBE'
directories = [pathlib.Path.cwd() / 'phenol_outfiles',pathlib.Path.cwd() / 'catechol_outfiles',
              pathlib.Path.cwd() / 'methylphenol_outfiles',pathlib.Path.cwd() / 'fluoroaniline_outfiles',
              pathlib.Path.cwd() / 'hydroxyaniline_outfiles']
mean_disp_elstner_10 = elstner_dispersion_collection(directories,functionals,snapshots_10)

In [44]:
# loading d2 d3 data
ligands = ['fluoroaniline','phenol','methylphenol','catechol','hydroxyaniline']
root_dir = 'dispersion/50_snaps/'
df_rel_mean_disp_10 = d2_d3_dispersion_correction(ligands,snapshots_10,mean_disp_elstner_10,root_dir)

### Uncorrected

In [53]:
functionals = ['PBE','VV10','B97M-V']
phenol = load_data(pathlib.Path.cwd() / 'phenol_outfiles', functionals, snapshots=snapshots_10)
catechol = load_data(pathlib.Path.cwd() / 'catechol_outfiles',functionals,snapshots=snapshots_10)
fluoroaniline = load_data( pathlib.Path.cwd() / 'fluoroaniline_outfiles', functionals,snapshots=snapshots_10)
hydroxyaniline = load_data( pathlib.Path.cwd() / 'hydroxyaniline_outfiles', functionals, snapshots=snapshots_10)

In [54]:
phenol_summary , phenol_data = load_data_all(pathlib.Path.cwd() / 'phenol_outfiles', functionals, snapshots=snapshots_10)


In [57]:
phenol_data['PBE']

,E,G_solv,G_solv_corrected,G_bind
24801,-29.874,3.349,3.349,-26.525
32401,-25.518,5.393,5.393,-20.126
17201,-26.917,2.975,2.975,-23.942
13401,-30.389,4.788,4.788,-25.601
21001,-28.263,4.192,4.192,-24.071
28601,-30.833,5.663,5.663,-25.170
9601,-26.244,3.802,3.802,-22.442
5801,-28.936,3.063,3.063,-25.873
36201,-27.513,2.678,2.678,-24.835
2001,-30.165,2.269,2.269,-27.896


In [244]:
# Special case for Methylphenol where some manual data manipulation is required due to restarted files
# Manual corrections to BLYP not currently needed but kept for the record
exclude_list_BLYP = ['complex_17201','host_17201','complex_24801','host_24801','complex_32401']
exclude_list_VV10 = ['complex_24801','host_24801','complex_32401','host_32401']
functionals = ['PBE','VV10','B97M-V']
outfile_dir_root = pathlib.Path.cwd() / 'methylphenol_outfiles'
list_of_mean_series = []
for functional in functionals:
    outfile_dir = outfile_dir_root / functional
    if functional=='BLYP':
        summary_df, mean_series = get_average_binding_manual(outfile_dir,correction=False,
                                                            exclude_function=exclude_function,
                                                            exclude_list=exclude_list_BLYP,
                                                            functional='BLYP',snapshots=snapshots_10)
    elif functional=='VV10':
        summary_df, mean_series = get_average_binding_manual(outfile_dir,correction=False,
                                                            exclude_function=exclude_function,
                                                            exclude_list=exclude_list_VV10,
                                                            functional='VV10',snapshots=snapshots_10)
    else:
        summary_df, mean_series = get_average_binding(outfile_dir,False,snapshots=snapshots_10)
        
    mean_series.name = functional
    list_of_mean_series.append(mean_series)
methylphenol = pd.concat(list_of_mean_series,axis=1)

In [245]:
normalization_series = - 5.6 - phenol.loc['G_bind':,:] # find normalization vecotr based on exp value of 5.6 for phenol
catechol_no_correction = catechol.loc['G_bind':,:] + normalization_series
methylphenol_no_correction = methylphenol.loc['G_bind':,:] + normalization_series
fluoroaniline_no_correction = fluoroaniline.loc['G_bind':,:] + normalization_series
hydroxyaniline_no_correction = hydroxyaniline.loc['G_bind':,:] + normalization_series

In [246]:
uncorrected_df_10 = catechol_no_correction.merge(methylphenol_no_correction,how='outer')
uncorrected_df_10 = uncorrected_df_10.merge(fluoroaniline_no_correction,how='outer')
uncorrected_df_10 = uncorrected_df_10.merge(hydroxyaniline_no_correction,how='outer')
uncorrected_df_10['Exp']=[-4.4,-4.4,-5.5,0.0]
uncorrected_df_10['5_snaps']=[-12.2,-10.1,-6.3,-8.2]
uncorrected_df_10.rename(index = {0:'catechol',1:"methylphenol",2:'fluoroaniline',3:'hydroxyaniline'},inplace=True)
uncorrected_df_10 = edit_pbe_df(uncorrected_df_10,df_rel_mean_disp_10)
uncorrected_df_10['MM']=mm_10_uncorrected
uncorrected_df_10

,PBE,VV10,B97M-V,Exp,5_snaps,PBE_bj,PBE_bjm,PBE_old,PBE_zero,MM
catechol,-10.769,-11.285,-10.848,-4.4,-12.2,-10.122,-10.119,-10.113,-10.157,-4.332
methylphenol,-8.713,-9.628,-8.649,-4.4,-10.1,-6.237,-6.229,-6.114,-6.245,-6.624
fluoroaniline,-6.692,-6.217,-6.867,-5.5,-6.3,-6.077,-6.084,-6.140,-6.050,-10.128
hydroxyaniline,-8.040,-8.558,-8.719,0.0,-8.2,-4.832,-4.821,-4.687,-4.840,-6.619


### Corrected, 10 snaps

In [247]:
functionals = ['PBE','VV10','B97M-V']
snapshots_10 = ['24801', '32401', '17201', '13401', '21001', '28601', '9601', '5801','36201','2001'] 
phenol = load_data(pathlib.Path.cwd() / 'phenol_outfiles', functionals,correction=True, snapshots=snapshots_10)
catechol = load_data(pathlib.Path.cwd() / 'catechol_outfiles',functionals,correction=True,snapshots=snapshots_10)
fluoroaniline = load_data( pathlib.Path.cwd() / 'fluoroaniline_outfiles', functionals,correction=True,snapshots=snapshots_10)
hydroxyaniline = load_data( pathlib.Path.cwd() / 'hydroxyaniline_outfiles', functionals,correction=True,snapshots=snapshots_10)

In [248]:
# Special case for Methylphenol where some manual data manipulation is required due to restarted files
# Manual corrections to BLYP not currently needed but kept for the record
exclude_list_BLYP = ['complex_17201','host_17201','complex_24801','host_24801','complex_32401']
exclude_list_VV10 = ['complex_24801','host_24801','complex_32401','host_32401']
functionals = ['PBE','VV10','B97M-V']
outfile_dir_root = pathlib.Path.cwd() / 'methylphenol_outfiles'
list_of_mean_series = []
for functional in functionals:
    outfile_dir = outfile_dir_root / functional
    if functional=='BLYP':
        summary_df, mean_series = get_average_binding_manual(outfile_dir,correction=True,
                                                            exclude_function=exclude_function,
                                                            exclude_list=exclude_list_BLYP,
                                                            functional='BLYP',snapshots=snapshots_10)
    elif functional=='VV10':
        summary_df, mean_series = get_average_binding_manual(outfile_dir,correction=True,
                                                            exclude_function=exclude_function,
                                                            exclude_list=exclude_list_VV10,
                                                            functional='VV10',snapshots=snapshots_10)
    else:
        summary_df, mean_series = get_average_binding(outfile_dir,True,snapshots=snapshots_10)
        
    mean_series.name = functional
    list_of_mean_series.append(mean_series)
methylphenol = pd.concat(list_of_mean_series,axis=1)

In [249]:
normalization_series = - 5.6 - phenol.loc['G_bind':,:] # find normalization vecotr based on exp value of 5.6 for phenol
catechol_correction = catechol.loc['G_bind':,:] + normalization_series
methylphenol_correction = methylphenol.loc['G_bind':,:] + normalization_series
fluoroaniline_correction = fluoroaniline.loc['G_bind':,:] + normalization_series
hydroxyaniline_correction = hydroxyaniline.loc['G_bind':,:] + normalization_series

In [252]:
corrected_df_10 = catechol_correction.merge(methylphenol_correction,how='outer')
corrected_df_10 = corrected_df_10.merge(fluoroaniline_correction,how='outer')
corrected_df_10 = corrected_df_10.merge(hydroxyaniline_correction,how='outer')
corrected_df_10['Exp']=[-4.4,-4.4,-5.5,0.0]
corrected_df_10['50_snaps']=[-9.0,-8.5,-5.9,-6.2]
corrected_df_10.rename(index = {0:'catechol',1:"methylphenol",2:'fluoroaniline',3:'hydroxyaniline'},inplace=True)
corrected_df_10 = edit_pbe_df(corrected_df_10,df_rel_mean_disp_10)
corrected_df_10['MM']=mm_10_corrected
corrected_df_10

,PBE,VV10,B97M-V,Exp,50_snaps,PBE_bj,PBE_bjm,PBE_old,PBE_zero,MM
catechol,-9.678,-10.045,-9.664,-4.4,-9.0,-9.031,-9.029,-9.022,-9.066,-4.276
methylphenol,-6.327,-6.721,-5.714,-4.4,-8.5,-3.851,-3.843,-3.728,-3.859,-6.524
fluoroaniline,-5.467,-4.742,-5.446,-5.5,-5.9,-4.852,-4.858,-4.915,-4.824,-10.032
hydroxyaniline,-5.351,-5.939,-5.983,0.0,-6.2,-2.143,-2.132,-1.998,-2.151,-6.579


# 5 Snaps

In [253]:
snapshots_5 = ['24801', '32401', '17201', '9601','2001'] 

## Dispersion, 5 snaps

In [254]:
# loading elstner data
functionals = 'PBE'
directories = [pathlib.Path.cwd() / 'phenol_outfiles',pathlib.Path.cwd() / 'catechol_outfiles',
              pathlib.Path.cwd() / 'methylphenol_outfiles',pathlib.Path.cwd() / 'fluoroaniline_outfiles',
              pathlib.Path.cwd() / 'hydroxyaniline_outfiles']
mean_disp_elstner_5 = elstner_dispersion_collection(directories,functionals,snapshots_5)

In [255]:
# loading d2 d3 data
ligands = ['fluoroaniline','phenol','methylphenol','catechol','hydroxyaniline']
root_dir = 'dispersion/50_snaps/'
df_rel_mean_disp_5 = d2_d3_dispersion_correction(ligands,snapshots_5,mean_disp_elstner_5,root_dir)

### Corrected, 5 snaps

In [256]:
functionals = ['PBE','VV10','B97M-V']
phenol = load_data(pathlib.Path.cwd() / 'phenol_outfiles', functionals,correction=True, snapshots=snapshots_5)
catechol = load_data(pathlib.Path.cwd() / 'catechol_outfiles',functionals,correction=True,snapshots=snapshots_5)
fluoroaniline = load_data( pathlib.Path.cwd() / 'fluoroaniline_outfiles', functionals,correction=True,snapshots=snapshots_5)
hydroxyaniline = load_data( pathlib.Path.cwd() / 'hydroxyaniline_outfiles', functionals,correction=True,snapshots=snapshots_5)

# Special case for Methylphenol where some manual data manipulation is required due to restarted files
# Manual corrections to BLYP not currently needed but kept for the record
exclude_list_BLYP = ['complex_17201','host_17201','complex_24801','host_24801','complex_32401']
exclude_list_VV10 = ['complex_24801','host_24801','complex_32401','host_32401']
functionals = ['PBE','VV10','B97M-V']
outfile_dir_root = pathlib.Path.cwd() / 'methylphenol_outfiles'
list_of_mean_series = []
for functional in functionals:
    outfile_dir = outfile_dir_root / functional
    if functional=='BLYP':
        summary_df, mean_series = get_average_binding_manual(outfile_dir,correction=True,
                                                            exclude_function=exclude_function,
                                                            exclude_list=exclude_list_BLYP,
                                                            functional='BLYP',snapshots=snapshots_5)
    elif functional=='VV10':
        summary_df, mean_series = get_average_binding_manual(outfile_dir,correction=True,
                                                            exclude_function=exclude_function,
                                                            exclude_list=exclude_list_VV10,
                                                            functional='VV10',snapshots=snapshots_5)
    else:
        summary_df, mean_series = get_average_binding(outfile_dir,True,snapshots=snapshots_5)
        
    mean_series.name = functional
    list_of_mean_series.append(mean_series)
methylphenol = pd.concat(list_of_mean_series,axis=1)

normalization_series = - 5.6 - phenol.loc['G_bind':,:] # find normalization vecotr based on exp value of 5.6 for phenol
catechol_correction = catechol.loc['G_bind':,:] + normalization_series
methylphenol_correction = methylphenol.loc['G_bind':,:] + normalization_series
fluoroaniline_correction = fluoroaniline.loc['G_bind':,:] + normalization_series
hydroxyaniline_correction = hydroxyaniline.loc['G_bind':,:] + normalization_series

corrected_df_5 = catechol_correction.merge(methylphenol_correction,how='outer')
corrected_df_5 = corrected_df_5.merge(fluoroaniline_correction,how='outer')
corrected_df_5 = corrected_df_5.merge(hydroxyaniline_correction,how='outer')
corrected_df_5['Exp']=[-4.4,-4.4,-5.5,0.0]
corrected_df_5['50_snaps']=[-9.0,-8.5,-5.9,-6.2]
corrected_df_5.rename(index = {0:'catechol',1:"methylphenol",2:'fluoroaniline',3:'hydroxyaniline'},inplace=True)
corrected_df_5 = edit_pbe_df(corrected_df_5,df_rel_mean_disp_5)
corrected_df_5['MM']=mm_5_corrected
corrected_df_5

,PBE,VV10,B97M-V,Exp,50_snaps,PBE_bj,PBE_bjm,PBE_old,PBE_zero,MM
catechol,-8.410,-9.174,-8.911,-4.4,-9.0,-6.913,-6.909,-6.876,-6.947,-5.086
methylphenol,-5.720,-6.524,-5.588,-4.4,-8.5,-2.114,-2.102,-1.939,-2.128,-6.994
fluoroaniline,-5.596,-4.939,-5.447,-5.5,-5.9,-5.126,-5.132,-5.190,-5.099,-10.128
hydroxyaniline,-4.309,-5.780,-6.227,0.0,-6.2,-0.089,-0.071,0.132,-0.101,-8.166


In [257]:
functionals = ['PBE','VV10','B97M-V']
phenol = load_data(pathlib.Path.cwd() / 'phenol_outfiles', functionals,correction=False, snapshots=snapshots_5)
catechol = load_data(pathlib.Path.cwd() / 'catechol_outfiles',functionals,correction=False,snapshots=snapshots_5)
fluoroaniline = load_data( pathlib.Path.cwd() / 'fluoroaniline_outfiles', functionals,correction=False,snapshots=snapshots_5)
hydroxyaniline = load_data( pathlib.Path.cwd() / 'hydroxyaniline_outfiles', functionals,correction=False,snapshots=snapshots_5)

# Special case for Methylphenol where some manual data manipulation is required due to restarted files
# Manual corrections to BLYP not currently needed but kept for the record
exclude_list_BLYP = ['complex_17201','host_17201','complex_24801','host_24801','complex_32401']
exclude_list_VV10 = ['complex_24801','host_24801','complex_32401','host_32401']
functionals = ['PBE','VV10','B97M-V']
outfile_dir_root = pathlib.Path.cwd() / 'methylphenol_outfiles'
list_of_mean_series = []
for functional in functionals:
    outfile_dir = outfile_dir_root / functional
    if functional=='BLYP':
        summary_df, mean_series = get_average_binding_manual(outfile_dir,correction=False,
                                                            exclude_function=exclude_function,
                                                            exclude_list=exclude_list_BLYP,
                                                            functional='BLYP',snapshots=snapshots_5)
    elif functional=='VV10':
        summary_df, mean_series = get_average_binding_manual(outfile_dir,correction=False,
                                                            exclude_function=exclude_function,
                                                            exclude_list=exclude_list_VV10,
                                                            functional='VV10',snapshots=snapshots_5)
    else:
        summary_df, mean_series = get_average_binding(outfile_dir,False,snapshots=snapshots_5)
        
    mean_series.name = functional
    list_of_mean_series.append(mean_series)
methylphenol = pd.concat(list_of_mean_series,axis=1)

normalization_series = - 5.6 - phenol.loc['G_bind':,:] # find normalization vecotr based on exp value of 5.6 for phenol
catechol_correction = catechol.loc['G_bind':,:] + normalization_series
methylphenol_correction = methylphenol.loc['G_bind':,:] + normalization_series
fluoroaniline_correction = fluoroaniline.loc['G_bind':,:] + normalization_series
hydroxyaniline_correction = hydroxyaniline.loc['G_bind':,:] + normalization_series

uncorrected_df_5 = catechol_correction.merge(methylphenol_correction,how='outer')
uncorrected_df_5 = uncorrected_df_5.merge(fluoroaniline_correction,how='outer')
uncorrected_df_5 = uncorrected_df_5.merge(hydroxyaniline_correction,how='outer')
uncorrected_df_5['Exp']=[-4.4,-4.4,-5.5,0.0]
uncorrected_df_5['50_snaps']=[-9.0,-8.5,-5.9,-6.2]
uncorrected_df_5.rename(index = {0:'catechol',1:"methylphenol",2:'fluoroaniline',3:'hydroxyaniline'},inplace=True)
uncorrected_df_5 = edit_pbe_df(uncorrected_df_5,df_rel_mean_disp_5)
uncorrected_df_5['MM']=mm_5_uncorrected
uncorrected_df_5

,PBE,VV10,B97M-V,Exp,50_snaps,PBE_bj,PBE_bjm,PBE_old,PBE_zero,MM
catechol,-11.517,-12.346,-12.182,-4.4,-9.0,-10.021,-10.017,-9.984,-10.055,-5.118
methylphenol,-9.704,-10.891,-9.961,-4.4,-8.5,-6.098,-6.085,-5.923,-6.111,-7.050
fluoroaniline,-6.475,-6.038,-6.484,-5.5,-5.9,-6.004,-6.011,-6.068,-5.978,-10.236
hydroxyaniline,-8.257,-9.507,-10.032,0.0,-6.2,-4.037,-4.019,-3.816,-4.049,-8.162
